# Econometrie 2 : Beauté et couleur de peau en politique 

## Code R pour tester les modèles 

## Préliminaires

### Données

In [7]:
setwd('C:/Users/Delanoue/Documents/ENSAE/S2-2A/Econométrie 2') #Pierre
#setwd('C:/Users/User/Desktop/Econométrie 2') #Maxime

data <- read.csv('data.csv', header = TRUE, sep=';')

n = dim(data)[1]
p = dim(data)[2]


#Bon format variables 
data$ab=as.numeric(gsub(",", ".", data$ab))
data$winner2<-as.factor(data$winner2)

#Creation Score de Beauté Pondéré 
norm_if39 = (data$if39-mean(data$if39))/sqrt(var(data$if39))
norm_im24 = (data$im24-mean(data$im24))/sqrt(var(data$im24))
norm_nif24= (data$nif24-mean(data$nif24))/sqrt(var(data$nif24))
norm_nim40 = (data$nim40-mean(data$nim40))/sqrt(var(data$nim40))

z_score = data.frame(norm_if39=norm_if39,norm_im24=norm_im24,norm_nif24=norm_nif24,norm_nim40=norm_nim40)

data$beaute_pond = data$ab*((z_score$norm_im24+z_score$norm_if39)/2)+(1-data$ab)*((z_score$norm_nim40+z_score$norm_nif24)/2)

### Libraries

In [8]:
library(stargazer)

library(MASS)



### Question 2

In [9]:
x_eval = data[,16:19]    #variables des évaluateurs
cor = cor(x_eval, method = c("pearson", "kendall", "spearman"))

### Question 3

In [10]:
moyenne_im24 = mean(data$im24)
moyenne_if39 = mean(data$if39)
moyenne_nif24 = mean(data$nif24)
moyenne_nim40 = mean(data$nim40)
variance_im24 = var(data$im24)
variance_if39 = var(data$if39)
variance_nif24= var(data$nif24)
variance_nim40 = var(data$nim40)

moyennes = c(moyenne_if39,moyenne_im24,moyenne_nif24,moyenne_nim40)
variances = c(variance_if39,variance_im24,variance_nif24,variance_nim40)

g_range <- range(0, moyennes, variances)
plot(moyennes, type="o", col="blue", ylim=g_range+1.5, axes=FALSE, ann=FALSE)

lines(variances, type="o", pch=22, lty=2, col="red")
axis(1, at=1:4, lab=c("if39","im24","nif24","nim40"))
axis(2, las=1, at=4*0:g_range[2])
box()
title(xlab="Evaluateurs", col.lab=rgb(0,0.5,0))
title(ylab="Valeurs", col.lab=rgb(0,0.5,0))
title(main="Moyenne et Variance", font.main=4)
legend(1, g_range[2]+1.5, c("moyennes","variances"), cex=0.8, 
       col=c("blue","red"), pch=21:22, lty=1:2)

#Normalisation

norm_if39 = (x_eval[,1]-moyenne_if39)/sqrt(variance_if39)
norm_im24 = (x_eval[,2]-moyenne_im24)/sqrt(variance_im24)
norm_nif24= (x_eval[,3]-moyenne_nif24)/sqrt(variance_nif24)
norm_nim40 = (x_eval[,4]-moyenne_nim40)/sqrt(variance_nim40)

z_score = data.frame(norm_if39=norm_if39,norm_im24=norm_im24,norm_nif24=norm_nif24,norm_nim40=norm_nim40)

ERROR: Error in png(tf, width, height, "in", pointsize, bg, res, antialias = antialias): impossible de démarrer le périphérique png()


Plot with title "Moyenne et Variance"

### Question 5

In [12]:
#Question 5 : Part de vote obtenus en fonction du score de beauté

#Création de la variable mean_score (moyenne des scores des 4 noteurs)
mean_score = rowSums(z_score)/dim(z_score)[1]
data=data.frame(data,mean_score)    #ajout de la nouvelle variable au dataframe
data$voteshare = as.numeric(data$voteshare)
data$mean_score=as.numeric(data$mean_score)

#Regression de voteshare sur mean_score
reg = lm(voteshare~mean_score,data = data)

#On affiche la regression
plot(x = mean_score,y = data[,'voteshare'],xlab = "score moyen", ylab = "voteshare")
title(main = "voteshare en fonction du score moyen")
abline(reg$coefficients[1],reg$coefficients[2])

#Séparation candidat sortant et opposant avec la variable incumbentcandidate (=1 si sortant)

data_sortant = subset(data,incumbentcandidate==1)
data_opposant = subset(data,incumbentcandidate==0)

#Régressions
reg1 = lm(voteshare~mean_score,data = data_sortant)
reg2 = lm(voteshare~mean_score,data = data_opposant)

#On affiche les regressions
par(mfrow = c(1,2))
#Candidats sortants
plot(x = data_sortant[,'mean_score'],y = data_sortant[,'voteshare'],xlab = "score moyen", ylab = "voteshare")
title(main = "Candidats sortants")
abline(reg1$coefficients[1],reg1$coefficients[2])

#Candidats opposants

plot(x = data_opposant[,'mean_score'],y = data_opposant[,'voteshare'],xlab = "score moyen", ylab = "voteshare")
title(main = "Candidats opposants")
abline(reg2$coefficients[1],reg2$coefficients[2])

#iii)

reg = lm(voteshare~skin,data = data)
reg_sortant = lm(voteshare~skin,data = data_sortant)
reg_opposant = lm(voteshare~skin,data = data_opposant)

par(mfrow = c(1,3))
#Tous les candidats
plot(x = data[,'skin'],y = data[,'voteshare'],xlab = "skin", ylab = "voteshare")
title(main = "Tous les candidats")
abline(reg$coefficients[1],reg$coefficients[2])

#Candidats sortants
plot(x = data_sortant[,'skin'],y = data_sortant[,'voteshare'],xlab = "skin", ylab = "voteshare")
title(main = "Candidats sortants")
abline(reg_sortant$coefficients[1],reg_sortant$coefficients[2])

#Candidats opposants

plot(x = data_opposant[,'skin'],y = data_opposant[,'voteshare'],xlab = "skin", ylab = "voteshare")
title(main = "Candidats opposants")
abline(reg_opposant$coefficients[1],reg_opposant$coefficients[2])

#iv)

ab = data[,'ab']
ab_sort = sort(ab)
#On fait un groupe 
data_low_ab = subset(data,ab<0.134)
data_high_ab = subset(data,ab>=0.134)

data_low_sortant = subset(data_low_ab,incumbentcandidate==1)
data_low_opposant = subset(data_low_ab,incumbentcandidate==0)
data_high_sortant = subset(data_high_ab,incumbentcandidate==1)
data_high_opposant = subset(data_high_ab,incumbentcandidate==0)

#Groupe low_ab

reg = lm(voteshare~skin,data = data_low_ab)
reg_sortant = lm(voteshare~skin,data = data_low_sortant)
reg_opposant = lm(voteshare~skin,data = data_low_opposant)

par(mfrow = c(1,3))
#Tous les candidats low ab
plot(x = data_low_ab[,'skin'],y = data_low_ab[,'voteshare'],xlab = "skin", ylab = "voteshare")
title(main = "Candidats faible ab")
abline(reg$coefficients[1],reg$coefficients[2])

#Candidats sortants low ab
plot(x = data_low_sortant[,'skin'],y = data_low_sortant[,'voteshare'],xlab = "skin", ylab = "voteshare")
title(main = "Candidats sortants faible ab")
abline(reg_sortant$coefficients[1],reg_sortant$coefficients[2])

#Candidats opposants low ab

plot(x = data_low_opposant[,'skin'],y = data_low_opposant[,'voteshare'],xlab = "skin", ylab = "voteshare")
title(main = "Candidats opposants faible ab")
abline(reg_opposant$coefficients[1],reg_opposant$coefficients[2])

#Groupe high_ab

reg = lm(voteshare~skin,data = data_high_ab)
reg_sortant = lm(voteshare~skin,data = data_high_sortant)
reg_opposant = lm(voteshare~skin,data = data_high_opposant)

par(mfrow = c(1,3))
#Tous les candidats high ab
plot(x = data_high_ab[,'skin'],y = data_high_ab[,'voteshare'],xlab = "skin", ylab = "voteshare")
title(main = "Candidats fort ab")
abline(reg$coefficients[1],reg$coefficients[2])

#Candidats sortants high ab
plot(x = data_high_sortant[,'skin'],y = data_high_sortant[,'voteshare'],xlab = "skin", ylab = "voteshare")
title(main = "Candidats sortants fort ab")
abline(reg_sortant$coefficients[1],reg_sortant$coefficients[2])

#Candidats opposants high ab

plot(x = data_high_opposant[,'skin'],y = data_high_opposant[,'voteshare'],xlab = "skin", ylab = "voteshare")
title(main = "Candidats opposants fort ab")
abline(reg_opposant$coefficients[1],reg_opposant$coefficients[2])


ERROR: Error in png(tf, width, height, "in", pointsize, bg, res, antialias = antialias): impossible de démarrer le périphérique png()


Plot with title "voteshare en fonction du score moyen"

ERROR: Error in png(tf, width, height, "in", pointsize, bg, res, antialias = antialias): impossible de démarrer le périphérique png()


Plot with title "Candidats opposants"

ERROR: Error in png(tf, width, height, "in", pointsize, bg, res, antialias = antialias): impossible de démarrer le périphérique png()


Plot with title "Candidats opposants"

ERROR: Error in png(tf, width, height, "in", pointsize, bg, res, antialias = antialias): impossible de démarrer le périphérique png()


Plot with title "Candidats opposants faible ab"

ERROR: Error in png(tf, width, height, "in", pointsize, bg, res, antialias = antialias): impossible de démarrer le périphérique png()


Plot with title "Candidats opposants fort ab"

### Question 7

In [15]:
data['true_voteshare'] = data['voteshare']*(1-0.0375)
data['Y'] = log(data['true_voteshare']/0.0375,2)

7-i)

In [40]:
reg1 = lm(Y~beaute_pond+incumbentcandidate+female+ballotorder,data = data)

reg2 = lm(Y~beaute_pond+incumbentcandidate+female+ballotorder+beaute_pond*incumbentcandidate,data = data)

reg3 = lm(Y~beaute_pond+incumbentcandidate+female+ballotorder+beaute_pond*female,data = data)

reg4 = lm(Y~skin+ab+incumbentcandidate+female+ballotorder,data = data)

reg5 = lm(Y~skin+ab+incumbentcandidate+female+ballotorder+skin*ab,data = data)

reg6 = lm(Y~beaute_pond+skin+ab+incumbentcandidate+female+ballotorder+skin*ab,data = data)

#ii)

reg7 = lm(Y~beaute_pond+skin+ab+skin*ab,data = data)

In [42]:
#stargazer(reg1)
#stargazer(reg2)
#stargazer(reg3)
#stargazer(reg4)
#stargazer(reg5)
#stargazer(reg6)
#stargazer(reg7)

### Question 8

In [ ]:
#Question 8
#ii)Test de corrélation entre beaute_pond et les variables physiologiques
reg = lm(beaute_pond~forehead+chin+nose+incumbentcandidate+female+ballotorder, data = data)
summary(reg)

#Test de Sargan
reg = lm(beaute_pond~forehead_norm+chin_norm+female+incumbentcandidate+ballotorder,data = data)
X = predict(object = reg, newdata = data)
data['instruments'] = X
reg_1 = lm(Y~instruments + incumbentcandidate + female + ballotorder ,data = data)
res = residuals(reg_1)
reg_u = lm(res~incumbentcandidate+ballotorder+female,data = data)
stargazer(reg_u)


#forehead
#On normalise forehead pour pouvoir le mettre au carré dans la régression
normalized = function(variable){ 
  moyenne = mean(variable)
  variance = mean(variable^2)-mean(variable)^2
  var_norm = (variable-moyenne)/sqrt(variance)
  var_norm
}
 
data['forehead_norm'] = normalized(as.numeric(data$forehead))
data['forehead_norm_squared'] = data['forehead_norm']^2
reg = lm(beaute_pond~forehead_norm^2,data = data)
summary(reg)

#nose 
data['nose_norm'] = normalized(as.numeric(data$nose))
data['nose_norm_squared'] = data['nose_norm']^2
reg = lm(beaute_pond~nose_norm^2,data = data)
summary(reg)

#chin
data['chin_norm'] = normalized(as.numeric(data$chin))
data['chin_norm_squared'] = data['chin_norm']^2
reg = lm(beaute_pond~chin_norm^2,data = data)
summary(reg)


#Les trois ensembles
reg = lm(beaute_pond~forehead_norm+nose_norm+chin_norm+female+incumbentcandidate+ballotorder,data = data)
stargazer(reg)

In [ ]:
#8-iii)
reg = lm(beaute_pond~forehead_norm+chin_norm+female+incumbentcandidate+ballotorder,data = data)
X = predict(object = reg, newdata = data)
data['instruments'] = X
reg_1 = lm(Y~instruments + incumbentcandidate + female + ballotorder ,data = data)
res = residuals(reg_1)
reg_u = lm(res~incumbentcandidate+ballotorder+female,data = data)
summary(reg_u)
stargazer(reg_1)

reg_2 = lm(Y~instruments+incumbentcandidate+female+ballotorder+instruments*incumbentcandidate,data = data)
stargazer(reg_2)

reg_3 = lm(Y~instruments+incumbentcandidate+female+ballotorder+instruments*female,data = data)
stargazer(reg_3)

reg_6 = lm(Y~instruments+skin*ab+skin+female+incumbentcandidate+ab+ballotorder,data = data)
summary(reg_6)

reg = lm(beaute_pond~skin*ab, data = data)
summary(reg)


### Question 9

In [43]:
logit1 <- glm(winner ~beaute_pond+incumbentcandidate+female+ballotorder,family=binomial(link='logit'),data=data)

logit2 <- glm(winner ~beaute_pond+incumbentcandidate+female+ballotorder+beaute_pond*incumbentcandidate,family=binomial(link='logit'),data=data)

logit3 <- glm(winner ~beaute_pond+incumbentcandidate+female+ballotorder+beaute_pond*female,family=binomial(link='logit'),data=data)

logit4 <- glm(winner ~skin+ab+incumbentcandidate+female+ballotorder,family=binomial(link='logit'),data=data)

logit5 <- glm(winner ~skin+ab+incumbentcandidate+female+ballotorder+skin*ab,family=binomial(link='logit'),data=data)

logit6 <- glm(winner ~beaute_pond+skin+ab+incumbentcandidate+female+ballotorder+skin*ab,family=binomial(link='logit'),data=data)

logit7 <- glm(winner ~beaute_pond+skin+ab+skin*ab,family=binomial(link='logit'),data=data)

In [47]:
#stargazer(logit1)
#stargazer(logit2)
#stargazer(logit3)
#stargazer(logit4)
#stargazer(logit5)
#stargazer(logit6)

#stargazer(logit7)

### Question 10

In [37]:
reg_preliminaire1 = lm(beaute_pond~ballotorder+incumbentcandidate+female+chin, data = data)
eta1 = residuals(reg_preliminaire1)
probit1 = glm(winner~beaute_pond+ballotorder+female+incumbentcandidate+eta1,data = data,family = binomial(link='probit'))

In [39]:
#stargazer(probit1)

### Question 11

In [16]:
multi1 <-polr(winner2 ~ beaute_pond+skin+ab+female+skin*ab+beaute_pond*female,data=data, Hess=TRUE)

In [ ]:
#stargazer(multi1)